In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
from scipy.stats import pearsonr
from calendar import month_abbr
import regionmask
import pandas as pd
import geopandas as gp
import scipy.stats as stats

# Import data

In [3]:
dpr_GFDL_E_regrid = xr.open_mfdataset('/scratch/x77/nxm561/SMILEs/GFDL-ESM2M/pr_regrid/pr_AmonOmon_GFDL-ESM2M_historical_rcp85_r*.nc',parallel=True,combine='nested',concat_dim = 'ens_number').pr.sel(time = slice('1921','2100'))
dtos_GFDL_E = xr.open_mfdataset('/scratch/x77/nxm561/SMILEs/GFDL-ESM2M/tos_regrid/tos_Omon_GFDL-ESM2M_historical_rcp85_r*.nc',parallel=True,combine='nested',concat_dim = 'ens_number').tos.sel(time = slice('1921','2100')).squeeze().compute()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 17.94 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


# First data processing

In [5]:
# # in terminal
# module load nco
# ncea tos_...*.nc 
# ensemblemean.nc

In [6]:
# ensemble mean
pr_GFDL_E_regrid_ens_mean = xr.load_dataset('/scratch/x77/nxm561/SMILEs/GFDL-ESM2M/pr_regrid/pr_AmonOmon_GFDL-ESM2M_historical_rcp85_ensemble_mean.nc').pr.sel(time = slice('1921','2100'))

# anomaly with respect to ensemble mean (run at the first time)
# pr_GFDL_E_regrid_anom = (dpr_GFDL_E_regrid - pr_GFDL_E_regrid_ens_mean).compute()
# pr_GFDL_E_regrid_anom.to_dataset(name='pr').to_netcdf(path='/g/data/x77/ah3693/Rainfall_risk_project/pr_GFDL_E_regrid_anom.nc')

In [7]:
pr_GFDL_E_regrid_anom = xr.load_dataset('/g/data/x77/ah3693/Rainfall_risk_project/pr_GFDL_E_regrid_anom.nc').pr

# Some universal things

In [8]:
# Weights for spatial average
weights = np.cos(np.deg2rad(dtos_GFDL_E.lat))
weights.name = "weights"

# Spatial mean timeseries calculation

## - Nino3.4 timeseries (NOAA ONI method)

In [9]:
def anom(df,time_period,base_year):
    gb = df.sel(time=time_period).groupby('time.month')
    gb_base = df.sel(time=base_year).groupby('time.month')
    df_anom = gb - gb_base.mean(dim='time')
    return df_anom
    
def nino34_index(tos_file):
    tos_nino34 = tos_file.where((tos_file.lat<=5) & (tos_file.lat>=-5) & (tos_file.lon>=190) & (tos_file.lon<=240), drop=True)
    
    weights = np.cos(np.deg2rad(tos_nino34.lat))
    weights.name = "weights"
    tos_weighted = tos_nino34.weighted(weights)
    weighted_mean = tos_weighted.mean(dim=['lat','lon'])
    
    anomalies = []
    
    # 1921-1935
    anomaly = anom(weighted_mean, slice(str(1921),str(1935)), slice(str(1921),str(1950)))
    anomalies.append(anomaly)

    # 1936-2090
    for year in range(1936,2090,5):
        time_period = slice(str(year),str(year+4))
        base_year = slice(str(year-15),str(year+14))
        
        anomaly = anom(weighted_mean,time_period,base_year)
        anomalies.append(anomaly)

    # 2091-2100
    anomaly = anom(weighted_mean, slice(str(2091),str(2100)), slice(str(2071),str(2100)))
    anomalies.append(anomaly)

    # Concatenate the anomalies along the time dimension
    all_anomalies = xr.concat(anomalies, dim='time')

    ## index = mean sst over the whole region
    # detrended_nino34 = all_anomalies.rolling(time = 3,center=True).mean().dropna(dim='time')
    # nino34_SON = detrended_nino34.sel(time = detrended_nino34.time.dt.month.isin([10]))
    nino34_S_O_N = all_anomalies.sel(time = all_anomalies.time.dt.month.isin([9, 10, 11]))

    return nino34_S_O_N #detrended_nino34, nino34_SON.sel(time=slice('1921','2000'))
    # return detrended_nino34.sel(time=slice('1921','2000')), nino34_S_O_N.sel(time=slice('1921','2000')) #, nino34_SON.sel(time=slice('1921','2000'))

In [10]:
# # (run at the first time)
# nino34_GFDL_E, nino34_GFDL_E_SON = nino34_index(dtos_GFDL_E) # .sel(ens_number = slice(0,30))
# nino34_GFDL_E_S_O_N = nino34_index(dtos_GFDL_E)

In [11]:
# (run at the first time)
# nino34_GFDL_E.to_dataset(name='tos').to_netcdf(path='/g/data/x77/ah3693/Rainfall_risk_project/nino34_GFDL_E.nc')
# nino34_GFDL_E_SON.to_dataset(name='tos').to_netcdf(path='/g/data/x77/ah3693/Rainfall_risk_project/nino34_GFDL_E_SON.nc')
# nino34_GFDL_E_S_O_N.to_dataset(name='tos').to_netcdf(path='/g/data/x77/ah3693/Rainfall_risk_project/nino34_GFDL_E_S_O_N.nc')

In [12]:
nino34_GFDL_E = xr.load_dataset('/g/data/x77/ah3693/Rainfall_risk_project/nino34_GFDL_E.nc').tos
nino34_GFDL_E_SON = xr.load_dataset('/g/data/x77/ah3693/Rainfall_risk_project/nino34_GFDL_E_SON.nc').tos
nino34_GFDL_E_S_O_N = xr.load_dataset('/g/data/x77/ah3693/Rainfall_risk_project/nino34_GFDL_E_S_O_N.nc').tos

In [ ]:
def select_ENSO_year(nino34):
    index_nino34 = nino34.round(1)
    
    threshold = 0.5
    
    # Single threshold
    mask_EN = (index_nino34 >= threshold) 
    mask_LN = (index_nino34 <= -threshold)

    # # Two threshold (for moderate-only analysis)
    # mask_EN = ((index_nino34 >= threshold) & (index_nino34 <= 1.5))
    # mask_LN = ((index_nino34 <= -threshold) & (index_nino34 >= -1.5))
    
    # Find the consecutive months with the threshold condition
    consecutive_months_EN = mask_EN.rolling(time=5, min_periods=5).sum() >= 5
    consecutive_months_LN = mask_LN.rolling(time=5, min_periods=5).sum() >= 5
    
    # Select the months that meet both the threshold and consecutive months condition
    selected_index_EN = index_nino34.where(consecutive_months_EN).dropna(dim='time')
    selected_index_LN = index_nino34.where(consecutive_months_LN).dropna(dim='time')
    
    elnino_y = selected_index_EN.sel(time = selected_index_EN.time.dt.month.isin([12]))
    elnino_y2 = selected_index_EN.sel(time = selected_index_EN.time.dt.month.isin([1,2,3,4]))
    lanina_y = selected_index_LN.sel(time = selected_index_LN.time.dt.month.isin([12]))
    lanina_y2 = selected_index_LN.sel(time = selected_index_LN.time.dt.month.isin([1,2,3,4]))
    
    elnino_year = np.sort(list(set(np.append(elnino_y.time.dt.year.values,elnino_y2.time.dt.year.values-1))))
    lanina_year = np.sort(list(set(np.append(lanina_y.time.dt.year.values,lanina_y2.time.dt.year.values-1))))
    
    return elnino_year, lanina_year

In [14]:
ens_num = len(nino34_GFDL_E)

# # (run at the first time)
# elnino_year_GFDL_E = [[] for _ in range(ens_num)]
# lanina_year_GFDL_E = [[] for _ in range(ens_num)]
# for i in range(ens_num):
#     elnino_year_GFDL_E[i], lanina_year_GFDL_E[i] = select_ENSO_year(nino34_GFDL_E.isel(ens_number = i))

In [15]:
# # (run at the first time)
# np.save('/g/data/x77/ah3693/Rainfall_risk_project/elnino_year_GFDL_E_med.npy', elnino_year_GFDL_E)
# np.save('/g/data/x77/ah3693/Rainfall_risk_project/lanina_year_GFDL_E_med.npy', lanina_year_GFDL_E)

In [16]:
elnino_year_GFDL_E = np.load('/g/data/x77/ah3693/Rainfall_risk_project/elnino_year_GFDL_E.npy',allow_pickle=True)
lanina_year_GFDL_E = np.load('/g/data/x77/ah3693/Rainfall_risk_project/lanina_year_GFDL_E.npy',allow_pickle=True)

In [17]:
elnino_year_GFDL_E_med = np.load('/g/data/x77/ah3693/Rainfall_risk_project/elnino_year_GFDL_E_med.npy',allow_pickle=True)
lanina_year_GFDL_E_med = np.load('/g/data/x77/ah3693/Rainfall_risk_project/lanina_year_GFDL_E_med.npy',allow_pickle=True)

## - Precipitation

In [19]:
# MDB precipitation
# NRM mask
NRM_clusters = '/g/data/w97/ah3693-2/Honours/NRM_clusters.zip'
NRM = gp.read_file('zip://'+NRM_clusters)
lon = pr_GFDL_E_regrid_anom.sel(lon = slice(110,160), lat = slice(-45,-5)).lon
lat = pr_GFDL_E_regrid_anom.sel(lon = slice(110,160), lat = slice(-45,-5)).lat
NRMmask = regionmask.mask_3D_geopandas(NRM,lon,lat)
NRM_precip_GFDL_E = pr_GFDL_E_regrid_anom.sel(lon = slice(110,160), lat = slice(-45,-5)).where(NRMmask)

In [20]:
MDB_precip_GFDL_E = NRM_precip_GFDL_E.sel(region = [0,2]).mean(dim=['lat','lon','region']).compute()

## SON three months rolling mean
# rolling_MDB_precip_GFDL_E = MDB_precip_GFDL_E.rolling(time = 3,center=True).mean().dropna(dim='time')
# MDB_precip_GFDL_E_SON = rolling_MDB_precip_GFDL_E.sel(time = rolling_MDB_precip_GFDL_E.time.dt.month.isin([10]))*1000*3600*24*91/3

# September, October, November single months all selected out
MDB_precip_GFDL_E_S_O_N = MDB_precip_GFDL_E.sel(time = MDB_precip_GFDL_E.time.dt.month.isin([9, 10, 11]))*3600*24*91/3